Playground
==

You can test the model here

In [ ]:
# build & run test
!./scripts/build_n_test.sh

In [ ]:
# Before playing, make sure you already build the project
import dummyml
import numpy as np
import cv2
from matplotlib import pyplot as plt

Naive Bayes Classifier
---

In [ ]:
dummyX = np.zeros((4,))
nb_classifier = dummyml.naive_bayes_classifier(0,0)

nb_classifier.load('model/myIRIS.dmy')
print("Result:",nb_classifier(dummyX))

K Means Clustering
---

In [ ]:
img = cv2.cvtColor(cv2.imread('./model/building.jpg'), cv2.COLOR_BGR2RGB)
lenna = np.array(img, dtype=np.float64)/256
plt.imshow(img)
plt.show()

In [ ]:
dataset_size = lenna.shape[0]*lenna.shape[1]
lennaX = lenna.flatten().reshape((dataset_size,3))
k = 5
pool = 30
clusters = [np.random.randint(0, k, size=dataset_size, dtype=np.int32) for p in range(pool)]
kms_pool = [dummyml.k_means(3,k) for i in range(pool)]

for iter in range(20):
    print(f"Iter #{iter}....")
    for p in range(pool):
        kms_pool[p].fit(lennaX, clusters[p])

inertias = [kms_pool[p].inertia for p in range(pool)]
min_index = np.argmin(inertias)
print("pool #",min_index,", min Inertia = ",inertias[min_index])

means = kms_pool[min_index].means()
print("Final means:\n",means)
flat = np.array([means[np.int8(cluster)] for cluster in clusters[min_index]]).flatten()
new_img = flat.reshape(lenna.shape[0],lenna.shape[1],3)
plt.imshow(new_img)
plt.show()


In [ ]:
data_points = np.random.random((150,2))
data_points[:50,:] += 2
data_points[50:100,:] += 1
clstr = np.random.randint(0, 3, size=150, dtype=np.int32)
kms = dummyml.k_means(2, 3)

for iter in range(20):
    kms.fit(data_points, clstr)

means = kms.means()

plt.scatter(data_points[:,0], data_points[:,1])
plt.scatter(means[:,0], means[:,1], color='red')
plt.show()

In [ ]:
kms_pool[min_index].save("model/kms_building_best.dmy")

In [ ]:
loaded_kms = dummyml.k_means("model/kms_building_best.dmy")
print(loaded_kms.means())

Gaussian Process
---

In [ ]:
def f(x):
    return 2 + x*0.5 - x**2 + x**3*0.2 + np.random.randn(len(x))*3
x = np.linspace(-5, 5, 30)
y = f(x)

In [ ]:
gp = dummyml.gaussian_process(
    x.reshape((x.size,1)),
    y
)

In [ ]:
gp = dummyml.gaussian_process(
    x.reshape((x.size,1)),
    y,
    0.2,
    dummyml.kernel.RadialBasisFunctionKernel
)

In [ ]:
plt.scatter(x,y)

gp_x = np.linspace(-5, 5, 300)
pred = [gp(_x) for _x in gp_x.reshape(300,1)]
gp_y = np.array([mv[0] for mv in pred])
conf95 = np.array([mv[1]**0.5 * 2 for mv in pred])
plt.plot(gp_x, gp_y)
plt.fill_between(gp_x, gp_y + conf95, gp_y - conf95, alpha = 0.2)


plt.show()

Kernel K Means
---

In [ ]:
img = cv2.cvtColor(cv2.imread('./model/island.png'), cv2.COLOR_BGR2RGB)
island = np.array(img, dtype=np.float64)/256
dataset_size = island.shape[0]*island.shape[1]
island_x = island.flatten().reshape((dataset_size,3))

In [ ]:
K = 6
termination_cond = 1.0

In [ ]:
kkms = dummyml.kernel_k_means(K, island_x, dummyml.kernel.RadialBasisFunctionKernel)
y = np.random.randint(0, K, size=dataset_size, dtype=np.int32)
pre_iner = np.inf
print("initial y is",y[:20])
while True:
    kkms.fit(y)
    iner = kkms.inertia
    print("Inertia is:", iner,", y is",y[:20])
    if abs(iner - pre_iner) < termination_cond:
        print("Convergence.")
        break
    pre_iner = iner

color_pool = np.random.randint(0,256, size=(K, 3))
new_island = np.array(
    [color_pool[k] for k in y[:dataset_size]],
    dtype=np.uint8
).reshape(island.shape[0], island.shape[1], 3)

plt.imshow(new_island)
plt.show()

In [ ]:
img = cv2.cvtColor(cv2.imread('./model/forest.jpg'), cv2.COLOR_BGR2RGB)
forest = np.array(img, dtype=np.float64)/256
dataset_size = forest.shape[0]*forest.shape[1]
forest_x = forest.flatten().reshape((dataset_size,3))

In [ ]:
kkms = dummyml.kernel_k_means(K, forest_x, dummyml.kernel.RadialBasisFunctionKernel)
y = np.random.randint(0, K, size=dataset_size, dtype=np.int32)
pre_iner = np.inf
while True:
    kkms.fit(y)
    iner = kkms.inertia
    print("Inertia is:", iner)
    if abs(iner - pre_iner) < termination_cond:
        print("Convergence.")
        break
    pre_iner = iner

color_pool = np.random.randint(0,256, size=(K, 3))
new_forest = np.array(
    [color_pool[k] for k in y],
    dtype=np.uint8
).reshape(forest.shape[0], forest.shape[1], 3)

plt.imshow(new_forest)
plt.show()